# Configuration Validator

This notebook validates Excel configuration files and optionally loads them to the database.

**You can do the following with this tool:**
- Validate configuration Excel file (without registering them)
- View detailed validation results per sheet
- Optionally register validated configuration to database (not recommended)
- Preview transformer outputs without creating batches and jobs
- Run any or all transformers to see job configurations

In [1]:
# Import required modules
from helpers import configuration, ux, cycle
from helpers.configuration import ConfigurationError
from helpers.database import execute_query
import pandas as pd
from pathlib import Path
import json

## Check Active Cycle

In [2]:
# Get and display active cycle
ux.header("Active Cycle Status")

active_cycle = cycle.get_active_cycle()
if active_cycle:
    ux.success(f"Active Cycle: {active_cycle['cycle_name']}")
    cycle_id = active_cycle['id']
    
    # Display cycle info
    info_data = [
        ['Cycle ID', cycle_id],
        ['Cycle Name', active_cycle['cycle_name']],
        ['Status', active_cycle['status']],
        ['Created', str(active_cycle['created_ts'])]
    ]
    ux.table(info_data, headers=['Property', 'Value'])
else:
    ux.error("No active cycle found. Please create a cycle first.")
    raise Exception("No active cycle")

Property,Value
Cycle ID,2
Cycle Name,Analysis-2025-Q4-v2
Status,ACTIVE
Created,2025-11-06 16:33:00.112120+00:00


## Select Configuration File

In [3]:
# File selection with validation
ux.subheader("Configuration File Selection")

def validate_excel_path(path_str):
    """Validate that path exists and is an Excel file"""
    if not path_str.strip():
        return False
    path = Path(path_str)
    if not path.exists():
        print(f"File not found: {path_str}")
        return False
    if path.suffix.lower() not in ['.xlsx', '.xls']:
        print("File must be an Excel file (.xlsx or .xls)")
        return False
    return True

excel_path = ux.text_input(
    "Enter path to Excel configuration file",
    placeholder="/home/jovyan/workspace/tests/files/valid_excel_configuration.xlsx",
    validation=validate_excel_path
)

if excel_path is None:
    ux.error("File selection cancelled")
    raise Exception("Cancelled")

# Display file info
file_path = Path(excel_path)
file_size_mb = file_path.stat().st_size / (1024 * 1024)
file_data = [
    ['File Path', str(file_path.absolute())],
    ['File Name', file_path.name],
    ['File Size', f"{file_size_mb:.2f} MB"],
    ['Last Modified', str(pd.Timestamp.fromtimestamp(file_path.stat().st_mtime))]
]
ux.table(file_data, headers=['Property', 'Value'])
ux.success("File found and ready for validation")

Enter path to Excel configuration file (e.g., /home/jovyan/workspace/tests/files/valid_excel_configuration.xlsx) (enter 'cancel' to stop)


>  /home/jovyan/workspace/tests/files/valid_excel_configuration.xlsx


Property,Value
File Path,/home/jovyan/workspace/tests/files/valid_excel_configuration.xlsx
File Name,valid_excel_configuration.xlsx
File Size,0.07 MB
Last Modified,2025-11-04 18:07:52.589818


## Validate Configuration

In [4]:
# Validate configuration without loading to database
ux.header("Configuration Validation")

try:
    validation_result = configuration.validate_configuration_file(
        cycle_id=cycle_id,
        excel_config_path=excel_path
    )
    
    if validation_result['validation_passed']:
        ux.success("✓ All validation checks passed!")
    else:
        ux.warning("⚠ Validation completed with warnings/errors")
    
    # Store for later use
    config_data = validation_result['configuration_data']
    validation_results = config_data.get('_validation', {})
    
except ConfigurationError as e:
    ux.error(f"Validation failed: {str(e)}")
    raise

### Validation Results Summary

In [5]:
# Display validation summary per sheet
ux.subheader("Sheet Validation Summary")

summary_data = []
for sheet_name, result in validation_results.items():
    if sheet_name == '_cross_sheet':
        continue
    
    status = result.get('status', 'UNKNOWN')
    error_count = len(result.get('errors', []))
    warning_count = len(result.get('warnings', []))
    row_count = result.get('row_count', 0)
    
    # Status indicator
    if status == 'SUCCESS':
        status_icon = '✓'
    elif status == 'ERROR':
        status_icon = '✗'
    else:
        status_icon = '?'
    
    summary_data.append([
        f"{status_icon} {sheet_name}",
        status,
        row_count,
        error_count,
        warning_count
    ])

summary_df = pd.DataFrame(summary_data, columns=['Sheet', 'Status', 'Rows', 'Errors', 'Warnings'])
ux.dataframe(summary_df)

# Check for cross-sheet validation
if '_cross_sheet' in validation_results:
    cross_result = validation_results['_cross_sheet']
    if cross_result['status'] == 'ERROR':
        ux.error(f"✗ Cross-Sheet Validation: {len(cross_result['errors'])} errors")
    else:
        ux.success("✓ Cross-Sheet Validation: Passed")

Sheet,Status,Rows,Errors,Warnings
✓ Metadata,SUCCESS,11,0,0
✓ Databases,SUCCESS,7,0,0
✓ Portfolios,SUCCESS,70,0,0
✓ Reinsurance Treaties,SUCCESS,3,0,0
✓ GeoHaz Thresholds,SUCCESS,121,0,0
✓ Analysis Table,SUCCESS,93,0,0
✓ Groupings,SUCCESS,96,0,0
✓ Products and Perils,SUCCESS,264,0,0
✓ Moody's Reference Data,SUCCESS,75,0,0


### Detailed Validation Results (Errors & Warnings)

In [6]:
# Show detailed errors and warnings
ux.subheader("Detailed Validation Results")

has_issues = False

for sheet_name, result in validation_results.items():
    errors = result.get('errors', [])
    warnings = result.get('warnings', [])
    
    if errors or warnings:
        has_issues = True
        print(f"\n{'='*60}")
        print(f"Sheet: {sheet_name}")
        print(f"{'='*60}")
        
        if errors:
            ux.error(f"Errors ({len(errors)}):")
            for i, error in enumerate(errors, 1):
                print(f"  {i}. {error}")
        
        if warnings:
            ux.warning(f"Warnings ({len(warnings)}):")
            for i, warning in enumerate(warnings, 1):
                print(f"  {i}. {warning}")

if not has_issues:
    ux.success("No errors or warnings found!")

## Preview Transformer Outputs

### Select Transformers to Run

In [10]:
# Get list of available transformers
ux.header("Transformer Preview")

if not validation_result['validation_passed']:
    ux.error("Cannot preview transformers with validation errors. Please fix errors first.")
else:
    available_transformers = configuration.get_transformer_list(include_test=False)
    
    ux.info(f"Available transformers: {len(available_transformers)}")
    
    # Display transformer options
    print("\nAvailable Transformers:")
    print("0. Run ALL transformers")
    for i, transformer in enumerate(available_transformers, 1):
        print(f"{i}. {transformer}")
    
    # Get user selection
    print("\nEnter transformer numbers (comma-separated) or 0 for all:")
    selection = input("> ").strip()
    
    # Parse selection
    selected_transformers = []
    if selection == '0':
        selected_transformers = available_transformers
        ux.success(f"Selected all {len(selected_transformers)} transformers")
    else:
        try:
            indices = [int(x.strip()) for x in selection.split(',')]
            for idx in indices:
                if 1 <= idx <= len(available_transformers):
                    selected_transformers.append(available_transformers[idx-1])
            ux.success(f"Selected {len(selected_transformers)} transformer(s)")
        except ValueError:
            ux.error("Invalid selection")
            selected_transformers = []


Available Transformers:
0. Run ALL transformers
1. EDM Creation
2. Portfolio Creation
3. MRI Import
4. Create Reinsurance Treaties
5. EDM DB Upgrade
6. GeoHaz
7. Portfolio Mapping
8. Analysis
9. Grouping
10. Export to RDM
11. Staging ETL

Enter transformer numbers (comma-separated) or 0 for all:


>  8


### Run Transformers and Display Results (Preview first 3)

In [11]:
selected_transformers

['Analysis']

In [14]:
# Run selected transformers
if selected_transformers:
    ux.subheader("Transformer Results")
    
    transformer_results = {}
    
    for transformer_name in selected_transformers:
        print(f"\n{'='*60}")
        print(f"Transformer: {transformer_name}")
        print(f"{'='*60}")
        
        try:
            job_configs = configuration.preview_transformer_jobs(
                batch_type=transformer_name,
                configuration_data=config_data
            )
            
            transformer_results[transformer_name] = job_configs
            
            ux.success(f"Generated {len(job_configs)} job configuration(s)")
            
            # Display preview of first 3 jobs
            preview_count = min(3, len(job_configs))
            if preview_count > 0:
                print(f"\nPreview (showing first {preview_count} of {len(job_configs)} jobs):")
                for i, job_config in enumerate(job_configs[:preview_count], 1):
                    print(f"\n  Job {i}:")
                    print(json.dumps(job_config, indent=2)) 
            
            if len(job_configs) > preview_count:
                ux.info(f"... and {len(job_configs) - preview_count} more job(s)")
                
        except ConfigurationError as e:
            ux.error(f"Failed to run transformer: {str(e)}")
            transformer_results[transformer_name] = None
    
    # Summary
    print(f"\n{'='*60}")
    ux.subheader("Transformer Summary")
    summary_data = []
    total_jobs = 0
    for transformer_name, jobs in transformer_results.items():
        if jobs is not None:
            job_count = len(jobs)
            total_jobs += job_count
            summary_data.append([transformer_name, job_count, '✓'])
        else:
            summary_data.append([transformer_name, 0, '✗'])
    
    summary_df = pd.DataFrame(summary_data, columns=['Transformer', 'Job Count', 'Status'])
    ux.dataframe(summary_df)
    ux.success(f"Total jobs across all transformers: {total_jobs}")
else:
    ux.info("No transformers selected")


Transformer: Analysis
<function transform_analysis at 0x75dc07de6200>



Preview (showing first 3 of 93 jobs):

  Job 1:
{
  "Metadata": {
    "Current Date Value": "202503",
    "EDM Data Version": "23.0.0",
    "Geocode Version": "23.0.0",
    "Hazard Version": "23.0.0",
    "DLM Model Version": 23,
    "Validate DLM Model Versions?": "Y",
    "Wildfire HD Model Version": 2,
    "SCS HD Model Version": 1,
    "Inland Flood HD Model Version": 1.2,
    "Validate HD Model Versions?": "Y",
    "Export RDM Name": "RMS_RDM_202503_QEM_USAP"
  },
  "Database": "RMS_EDM_202503_Quarterly_CBAP",
  "Portfolio": "CBHU",
  "Analysis Name": "CBHU_LT",
  "Analysis Profile": "DLM CBHU v23",
  "Output Profile": "Modeling_Output_Profile",
  "Event Rate": "RMS 2023 Historical Event Rates",
  "Reinsurance Treaty 1": "XPR_1_95",
  "Reinsurance Treaty 2": "XPR_1_100",
  "Reinsurance Treaty 3": null,
  "Reinsurance Treaty 4": null,
  "Reinsurance Treaty 5": null,
  "Tag 1": "202503_Quarterly_CBHU_LT",
  "Tag 2": null,
  "Tag 3": null,
  "Tag 4": null,
  "Tag 5": null
}

  Job 2

Transformer,Job Count,Status
Analysis,93,✓


### View Full Job Configuration

In [7]:
# Option to view full job configurations
if transformer_results and ux.yes_no("Do you want to see full job configurations?"):
    transformer_name = ux.dropdown(
        list(transformer_results.keys()),
        prompt="Select transformer to view full job configs"
    )
    
    if transformer_name and transformer_results[transformer_name]:
        jobs = transformer_results[transformer_name]
        ux.subheader(f"Full Job Configurations for: {transformer_name}")
        ux.info(f"Total jobs: {len(jobs)}")
        
        for i, job_config in enumerate(jobs, 1):
            print(f"\n{'='*60}")
            print(f"Job {i} of {len(jobs)}")
            print(f"{'='*60}")
            ux.json(job_config)
            
            # Pause after every 5 jobs
            if i % 5 == 0 and i < len(jobs):
                if not ux.yes_no(f"Continue viewing? ({len(jobs) - i} remaining)"):
                    break
else:
    ux.info("Skipped full job configuration view")

NameError: name 'transformer_results' is not defined

---

## Load Configuration to Database

In [ ]:
# Ask user if they want to load the configuration
ux.header("Load Configuration to Database")

if not validation_result['validation_passed']:
    ux.error("Cannot load configuration with validation errors. Please fix errors first.")
else:
    # Check for existing configurations
    query = "SELECT id, configuration_file_name, status FROM irp_configuration WHERE cycle_id = %s"
    existing_configs = execute_query(query, (cycle_id,))
    
    if not existing_configs.empty:
        ux.warning(f"Found {len(existing_configs)} existing configuration(s) for this cycle:")
        ux.dataframe(existing_configs)
        ux.warning("Loading a new configuration will DELETE all existing configurations for this cycle.")
    
    if ux.yes_no("Do you want to load this configuration to the database?"):
        try:
            config_id = configuration.load_configuration_file(
                cycle_id=cycle_id,
                excel_config_path=excel_path
            )
            ux.success(f"Configuration loaded successfully! Configuration ID: {config_id}")
            
            # Display loaded configuration info
            query = "SELECT id, configuration_file_name, status, created_ts FROM irp_configuration WHERE id = %s"
            config_info = execute_query(query, (config_id,))
            ux.dataframe(config_info)
            
        except ConfigurationError as e:
            ux.error(f"Failed to load configuration: {str(e)}")
    else:
        ux.info("Configuration not loaded. Validation results are still available for review.")

Do you want to load this configuration to the database? (y/n):  n


---